### Create a dataframe from Wikipedia page 

In [1]:
import pandas as pd # library for data analsysis
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import urllib.request # library to handle requests
import requests # library to handle requests

import numpy as np # library to handle data in a vectorized manner
import json # library to handle JSON files

# import k-means from clustering stage
from sklearn.cluster import KMeans

import seaborn as sns # plotting tools
import matplotlib.pyplot as plt # plotting tools
import folium # map rendering library

from bs4 import BeautifulSoup

### Use of Pandas read Wikipedia page table

In [2]:
# Use pandas.read_html to obtain table from Wikipedia
tables = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', header=0)

# Save the first table as dataframe, df
df = tables[0]

# Print the shape of original daraframe
print('Shape of df: ' + str(df.shape))
df.head()

Shape of df: (287, 3)


,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Use of web scraper for Wikipedia page table

In [3]:
# Download 'List of postal codes of Canada: M' as 'toronto_zipcode.html'
# Extract heads and columns and writing into 'toronto_zipcode.txt'

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
req = urllib.request.urlopen(url)
article = req.read().decode()

with open('toronto_zipcode.html', 'w') as fo:
    fo.write(article)

# Load article, turn into soup and get the <table>s.
article = open('toronto_zipcode.html').read()
soup = BeautifulSoup(article, 'html.parser')
tables = soup.find_all('table', class_='sortable')

# Search through the tables for the one with the headings
for table in tables:
    ths = table.find_all('th')
    headings = [th.text.strip() for th in ths]
    if headings[:3] == ['Postcode', 'Borough', 'Neighbourhood']:
        break

# Extract the columns we want and write to a semicolon-delimited text file.
with open('toronto_zipcode.txt', 'w') as fo:
    for tr in table.find_all('tr'):
        tds = tr.find_all('td')
        if not tds:
            continue
        Postcode, Borough, Neighbourhood = [td.text.strip() for td in tds[:3]]
        print('; '.join([Postcode, Borough, Neighbourhood]), file=fo)

# Convert to DataFrame and name the columns
df = pd.read_csv('toronto_zipcode.txt', sep=";", header=None)
df.columns = ['Postcode', 'Borough', 'Neighbourhood']

# Print the shape of original daraframe
print('Shape of df: ' + str(df.shape))
df.head()

Shape of df: (287, 3)


,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [4]:
# Select rows with only Borough assigned
not_assigned = df.iloc[0,1]
df1 = df[df['Borough'] != not_assigned].reset_index(drop=True)

# Set not assigned neighbourhood to match borough
df1['Neighbourhood'] = df1['Neighbourhood'].apply(lambda x: " Queen's Park" if x == not_assigned else x)

# Combine boroughs and neighbourhoods with the same postcode and separated with comma ','
df1 = df1.groupby(['Postcode'], sort=False).agg( ','.join).reset_index()

# Shape of modified data frame
print('Shape of df: ' + str(df1.shape))
df1.head()

Shape of df: (103, 3)


,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,"North York, North York","Lawrence Heights, Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park
